In [1]:
# Cell 1: Install Required Libraries

# Install osmnx if not available
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

# Install required packages
packages = ['osmnx', 'geopandas', 'shapely']
for pkg in packages:
    try:
        __import__(pkg)
        print(f"✓ {pkg} already installed")
    except ImportError:
        print(f"Installing {pkg}...")
        install_package(pkg)
        print(f"✓ {pkg} installed")

print("\n✅ All packages ready!")

✓ osmnx already installed
✓ geopandas already installed
✓ shapely already installed

✅ All packages ready!


In [3]:
# Cell 2: Import Libraries and Load Location Data

import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.ops import nearest_points
import warnings
import time
import os

warnings.filterwarnings('ignore')

# Configure OSMnx
ox.settings.log_console = False
ox.settings.use_cache = True
ox.settings.timeout = 180

print("Libraries imported successfully!")
print(f"OSMnx version: {ox.__version__}")

# Load the air quality dataset to get unique locations
aq_file = "data/processed/india_aq_transformed_last30days.csv"

if os.path.exists(aq_file):
    aq_df = pd.read_csv(aq_file)
    
    # Get unique locations
    locations_df = aq_df[['state', 'district', 'location_id', 'location_name', 'latitude', 'longitude']].drop_duplicates()
    locations_df = locations_df.reset_index(drop=True)
    
    print(f"\n📊 Air Quality Dataset loaded: {len(aq_df):,} records")
    print(f"📍 Unique monitoring locations: {len(locations_df)}")
    print(f"\n" + "="*60)
    print("Locations to extract physical features for:")
    print("="*60)
    print(locations_df[['state', 'district', 'location_name', 'latitude', 'longitude']].to_string())
else:
    print(f"❌ File not found: {aq_file}")
    locations_df = pd.DataFrame()

Libraries imported successfully!
OSMnx version: 2.0.7

📊 Air Quality Dataset loaded: 113,878 records
📍 Unique monitoring locations: 51

Locations to extract physical features for:
                state            district                                    location_name   latitude  longitude
0      Andhra Pradesh            Tirupati                       Tirumala, Tirupati - APPCB  13.670000  79.350000
1      Andhra Pradesh          Vijayawada                   Secretariat, Amaravati - APPCB  16.515083  80.518167
2      Andhra Pradesh       Visakhapatnam           GVM Corporation, Visakhapatnam - APPCB  17.722682  83.308197
3   Arunachal Pradesh            Itanagar                  Naharlagun, Naharlagun - APSPCB  27.103358  93.679645
4               Assam             Silchar                          Tarapur, Silchar - PCBA  24.828270  92.795250
5               Bihar           Bhagalpur                      Mayaganj, Bhagalpur - BSPCB  25.265194  87.012947
6               Bihar        

In [5]:
# Cell 3: Define Physical Feature Extraction Functions

# Search radius in meters for feature extraction
SEARCH_RADIUS = 2000  # 2 km radius

# Define OSM tags for each feature category based on project requirements
OSM_TAGS = {
    # Roads - for vehicular pollution source
    'roads': {
        'highway': ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 
                    'motorway_link', 'trunk_link', 'primary_link', 'secondary_link']
    },
    
    # Industrial zones - for industrial pollution source
    'industrial': {
        'landuse': ['industrial'],
        'building': ['industrial', 'factory', 'warehouse'],
        'man_made': ['works', 'chimney']
    },
    
    # Dump sites/Landfills - for waste-related pollution
    'dump_sites': {
        'landuse': ['landfill'],
        'amenity': ['waste_disposal', 'waste_transfer_station', 'recycling'],
        'man_made': ['wastewater_plant']
    },
    
    # Agricultural fields - for agricultural burning pollution
    'agricultural': {
        'landuse': ['farmland', 'farm', 'farmyard', 'orchard', 'vineyard', 
                    'plant_nursery', 'greenhouse_horticulture']
    },
    
    # Additional: Power plants (can contribute to pollution)
    'power_plants': {
        'power': ['plant', 'generator'],
        'man_made': ['chimney']
    },
    
    # Additional: Residential areas (for context)
    'residential': {
        'landuse': ['residential']
    },
    
    # Additional: Commercial areas
    'commercial': {
        'landuse': ['commercial', 'retail']
    }
}

def get_features_from_osm(lat, lon, tags, radius=SEARCH_RADIUS):
    """
    Extract features from OpenStreetMap for a given location.
    Returns GeoDataFrame of features or empty GeoDataFrame if none found.
    """
    try:
        # Create a point and buffer for the search area
        gdf = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        return gdf
    except Exception as e:
        # Return empty GeoDataFrame if no features found
        return gpd.GeoDataFrame()

def calculate_min_distance(point, features_gdf):
    """
    Calculate minimum distance from a point to nearest feature.
    Returns distance in meters.
    """
    if features_gdf.empty:
        return np.nan
    
    try:
        # Project to UTM for accurate distance calculation
        features_gdf_proj = features_gdf.to_crs(epsg=32643)  # UTM zone 43N for India
        point_proj = gpd.GeoSeries([point], crs='EPSG:4326').to_crs(epsg=32643).iloc[0]
        
        # Calculate distances
        distances = features_gdf_proj.geometry.distance(point_proj)
        return distances.min()
    except Exception as e:
        return np.nan

def count_features_in_radius(features_gdf, radius=SEARCH_RADIUS):
    """
    Count number of features within the search radius.
    """
    if features_gdf.empty:
        return 0
    return len(features_gdf)

def calculate_total_road_length(roads_gdf):
    """
    Calculate total length of roads in meters.
    """
    if roads_gdf.empty:
        return 0
    
    try:
        # Project to UTM for accurate length calculation
        roads_proj = roads_gdf.to_crs(epsg=32643)
        total_length = roads_proj.geometry.length.sum()
        return total_length
    except:
        return 0

def calculate_feature_area(features_gdf):
    """
    Calculate total area of polygon features in square meters.
    """
    if features_gdf.empty:
        return 0
    
    try:
        # Project to UTM for accurate area calculation
        features_proj = features_gdf.to_crs(epsg=32643)
        # Filter only polygons
        polygons = features_proj[features_proj.geometry.type.isin(['Polygon', 'MultiPolygon'])]
        if polygons.empty:
            return 0
        return polygons.geometry.area.sum()
    except:
        return 0

print("✅ Feature extraction functions defined!")
print(f"\nSearch radius: {SEARCH_RADIUS} meters ({SEARCH_RADIUS/1000} km)")
print(f"\nFeature categories to extract:")
for category in OSM_TAGS.keys():
    print(f"  • {category}")

✅ Feature extraction functions defined!

Search radius: 2000 meters (2.0 km)

Feature categories to extract:
  • roads
  • industrial
  • dump_sites
  • agricultural
  • power_plants
  • residential
  • commercial


In [7]:
# Cell 4: Test Extraction for a Single Location

if len(locations_df) > 0:
    # Select first location for testing
    test_loc = locations_df.iloc[0]
    test_lat = test_loc['latitude']
    test_lon = test_loc['longitude']
    
    print("="*70)
    print("TESTING PHYSICAL FEATURE EXTRACTION FOR SINGLE LOCATION")
    print("="*70)
    print(f"\n📍 Location: {test_loc['location_name']}")
    print(f"   State: {test_loc['state']}")
    print(f"   District: {test_loc['district']}")
    print(f"   Coordinates: ({test_lat}, {test_lon})")
    print(f"   Search radius: {SEARCH_RADIUS}m")
    
    # Create point geometry
    test_point = Point(test_lon, test_lat)
    
    # Extract each feature type
    test_results = {}
    
    print("\n" + "-"*50)
    print("Extracting features...")
    print("-"*50)
    
    for category, tags in OSM_TAGS.items():
        print(f"  Extracting {category}...", end=" ")
        
        features = get_features_from_osm(test_lat, test_lon, tags, SEARCH_RADIUS)
        count = count_features_in_radius(features)
        min_dist = calculate_min_distance(test_point, features)
        
        if category == 'roads':
            total_length = calculate_total_road_length(features)
            test_results[f'{category}_count'] = count
            test_results[f'{category}_min_distance_m'] = min_dist
            test_results[f'{category}_total_length_m'] = total_length
            print(f"✓ {count} features, min_dist={min_dist:.0f}m, length={total_length:.0f}m" if not np.isnan(min_dist) else f"✓ {count} features")
        else:
            total_area = calculate_feature_area(features)
            test_results[f'{category}_count'] = count
            test_results[f'{category}_min_distance_m'] = min_dist
            test_results[f'{category}_area_sqm'] = total_area
            print(f"✓ {count} features, min_dist={min_dist:.0f}m, area={total_area:.0f}sqm" if not np.isnan(min_dist) else f"✓ {count} features")
        
        time.sleep(0.5)  # Rate limiting
    
    print("\n" + "-"*50)
    print("Test Results Summary:")
    print("-"*50)
    for key, value in test_results.items():
        if isinstance(value, float) and not np.isnan(value):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")
    
    print("\n✅ Single location test completed successfully!")
else:
    print("❌ No locations available for testing")

TESTING PHYSICAL FEATURE EXTRACTION FOR SINGLE LOCATION

📍 Location: Tirumala, Tirupati - APPCB
   State: Andhra Pradesh
   District: Tirupati
   Coordinates: (13.67, 79.35)
   Search radius: 2000m

--------------------------------------------------
Extracting features...
--------------------------------------------------
  Extracting roads... ✓ 67 features, min_dist=64m, length=52878m
  Extracting industrial... ✓ 2 features, min_dist=397m, area=7470sqm
  Extracting dump_sites... ✓ 3 features, min_dist=976m, area=47901sqm
  Extracting agricultural... ✓ 0 features
  Extracting power_plants... ✓ 2 features, min_dist=353m, area=0sqm
  Extracting residential... ✓ 15 features, min_dist=214m, area=218283sqm
  Extracting commercial... ✓ 10 features, min_dist=446m, area=1533015sqm

--------------------------------------------------
Test Results Summary:
--------------------------------------------------
  roads_count: 67
  roads_min_distance_m: 63.78
  roads_total_length_m: 52878.48
  industri

In [9]:
# Cell 5: Define Main Feature Extraction Function for All Locations

def extract_physical_features_for_location(row):
    """
    Extract all physical features for a single location.
    Returns a dictionary with all extracted features.
    """
    lat = row['latitude']
    lon = row['longitude']
    point = Point(lon, lat)
    
    # Initialize result with location identifiers
    result = {
        'state': row['state'],
        'district': row['district'],
        'location_id': row['location_id'],
        'location_name': row['location_name'],
        'latitude': lat,
        'longitude': lon
    }
    
    # Extract features for each category
    for category, tags in OSM_TAGS.items():
        try:
            features = get_features_from_osm(lat, lon, tags, SEARCH_RADIUS)
            count = count_features_in_radius(features)
            min_dist = calculate_min_distance(point, features)
            
            result[f'{category}_count'] = count
            result[f'{category}_distance_m'] = round(min_dist, 2) if not np.isnan(min_dist) else np.nan
            
            if category == 'roads':
                result[f'{category}_total_length_m'] = round(calculate_total_road_length(features), 2)
            else:
                result[f'{category}_area_sqm'] = round(calculate_feature_area(features), 2)
                
        except Exception as e:
            result[f'{category}_count'] = 0
            result[f'{category}_distance_m'] = np.nan
            if category == 'roads':
                result[f'{category}_total_length_m'] = 0
            else:
                result[f'{category}_area_sqm'] = 0
        
        time.sleep(0.3)  # Rate limiting for OSM API
    
    return result

print("✅ Main extraction function defined!")
print("\nFeatures to be extracted per location:")
print("-"*50)
sample_features = ['location_id', 'location_name', 'latitude', 'longitude']
for category in OSM_TAGS.keys():
    sample_features.append(f'{category}_count')
    sample_features.append(f'{category}_distance_m')
    if category == 'roads':
        sample_features.append(f'{category}_total_length_m')
    else:
        sample_features.append(f'{category}_area_sqm')

for feat in sample_features:
    print(f"  • {feat}")

✅ Main extraction function defined!

Features to be extracted per location:
--------------------------------------------------
  • location_id
  • location_name
  • latitude
  • longitude
  • roads_count
  • roads_distance_m
  • roads_total_length_m
  • industrial_count
  • industrial_distance_m
  • industrial_area_sqm
  • dump_sites_count
  • dump_sites_distance_m
  • dump_sites_area_sqm
  • agricultural_count
  • agricultural_distance_m
  • agricultural_area_sqm
  • power_plants_count
  • power_plants_distance_m
  • power_plants_area_sqm
  • residential_count
  • residential_distance_m
  • residential_area_sqm
  • commercial_count
  • commercial_distance_m
  • commercial_area_sqm


In [11]:
# Cell 6: Extract Physical Features for ALL Locations

if len(locations_df) > 0:
    print("="*70)
    print("EXTRACTING PHYSICAL FEATURES FOR ALL LOCATIONS")
    print("="*70)
    print(f"\nTotal locations to process: {len(locations_df)}")
    print(f"Search radius: {SEARCH_RADIUS} meters")
    print(f"Estimated time: ~{len(locations_df) * 3} seconds\n")
    
    # Store all results
    all_physical_features = []
    
    # Process each location
    for idx, row in locations_df.iterrows():
        print(f"[{idx+1}/{len(locations_df)}] Processing: {row['location_name'][:40]}...", end=" ")
        
        try:
            features = extract_physical_features_for_location(row)
            all_physical_features.append(features)
            
            # Show summary
            roads = features.get('roads_count', 0)
            industrial = features.get('industrial_count', 0)
            agricultural = features.get('agricultural_count', 0)
            print(f"✓ Roads:{roads}, Industrial:{industrial}, Agricultural:{agricultural}")
            
        except Exception as e:
            print(f"✗ Error: {str(e)[:30]}")
            # Add row with NaN values
            error_result = {
                'state': row['state'],
                'district': row['district'],
                'location_id': row['location_id'],
                'location_name': row['location_name'],
                'latitude': row['latitude'],
                'longitude': row['longitude']
            }
            all_physical_features.append(error_result)
    
    # Convert to DataFrame
    physical_features_df = pd.DataFrame(all_physical_features)
    
    print("\n" + "="*70)
    print("EXTRACTION COMPLETE!")
    print("="*70)
    print(f"\n📊 Total locations processed: {len(physical_features_df)}")
    print(f"📋 Total features extracted: {len(physical_features_df.columns)}")
    
    print("\n" + "-"*50)
    print("Physical Features DataFrame Preview:")
    print("-"*50)
    print(physical_features_df.head(10))
else:
    print("❌ No locations to process")
    physical_features_df = pd.DataFrame()

EXTRACTING PHYSICAL FEATURES FOR ALL LOCATIONS

Total locations to process: 51
Search radius: 2000 meters
Estimated time: ~153 seconds

[1/51] Processing: Tirumala, Tirupati - APPCB... ✓ Roads:67, Industrial:2, Agricultural:0
[2/51] Processing: Secretariat, Amaravati - APPCB... ✓ Roads:52, Industrial:0, Agricultural:4
[3/51] Processing: GVM Corporation, Visakhapatnam - APPCB... ✓ Roads:393, Industrial:10, Agricultural:0
[4/51] Processing: Naharlagun, Naharlagun - APSPCB... ✓ Roads:37, Industrial:0, Agricultural:0
[5/51] Processing: Tarapur, Silchar - PCBA... ✓ Roads:74, Industrial:0, Agricultural:0
[6/51] Processing: Mayaganj, Bhagalpur - BSPCB... ✓ Roads:19, Industrial:1, Agricultural:2
[7/51] Processing: IGSC Planetarium Complex, Patna - BSPCB... ✓ Roads:357, Industrial:3, Agricultural:0
[8/51] Processing: Civic Center, Bhilai - Bhilai Steel Plan... ✓ Roads:137, Industrial:2, Agricultural:0
[9/51] Processing: Mangala, Bilaspur - CECB... ✓ Roads:117, Industrial:1, Agricultural:0
[10/5